In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
PATH ='C:/Users/ckdrj/Google 드라이브/baseball/data/total/'
#PATH = 'C:/Users/LIM CHANG GEON/Google 드라이브/baseball/data/total'
hiter16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv")
pitcher16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv")

In [3]:
def prepro(TEAM_ID,pitcher_year,hiter16):
    pitcher = pitcher_year[pitcher_year.T_ID==TEAM_ID].reset_index(drop=True)
    pitcher['index_date']=pd.date_range('2016-04-01', periods=144, freq='D')
    pitcher = pitcher.set_index('index_date')
    pitcher.WLS = pitcher.WLS.map({'W':1,'D':0.5,"L":0})
    pitcher["ERA"] = 27*pitcher.ER/pitcher.INN2
    pitcher["FIP"] = ((13*pitcher.HR+3*(pitcher.BB+pitcher.HP-pitcher.IB)-2*pitcher.KK)/(pitcher.INN2/3)) +3.2
    pitcher["BABIP"] = (pitcher.HIT-pitcher.HR)/(pitcher.AB-pitcher.KK-pitcher.HR+pitcher.SF)
    pitcher = pitcher[['G_ID','ERA','FIP','BABIP','KK','R','WLS']]
    
    return pitcher

In [4]:

#scaler_target = MinMaxScaler(feature_range=(0, 1))
#signal_data = scaler_va.fit_transform(pitcher16.iloc[:,:-1])
#target_minmax=scaler_target.fit_transform(pitcher16[["R"]])


In [5]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i + target_size])
        else:
            labels.append(target[i:i + target_size])
    return np.array(data), np.array(labels)

In [6]:
def split_data(pitcher,look_back = 10):
    future_target = 0
    TRAIN_SPLIT=120
    STEP=1
    scaler_va = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler_va.fit_transform(pitcher)
    x_train_single, y_train_single = multivariate_data(dataset[:,:], dataset[:,-1], 0, TRAIN_SPLIT, look_back,
                                                       future_target,STEP, single_step=True)
    x_val_single, y_val_single = multivariate_data(dataset[:,:], dataset[:,-1], TRAIN_SPLIT, None, look_back,
                                                   future_target,STEP, single_step=True)

    print('Single window of past history : {}'.format(x_train_single[0].shape))
    return x_train_single, y_train_single, x_val_single, y_val_single


In [7]:
def run(x_train_single, y_train_single, x_val_single, y_val_single, box):
    model = Sequential()
    model.add(Bidirectional(LSTM(box, input_shape=x_train_single.shape[-2:]))) #(timestep,feature)
#model.add(Dropout(0.1))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')

    early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)

    model.fit(x_train_single, y_train_single, epochs=100,
          batch_size=3, verbose=1, callbacks=[early_stop])
    pre = model.predict(x_val_single)
    score = model.evaluate(x_val_single,y_val_single)
    answer = y_val_single
    
    return score, answer, pre

In [15]:
TEAM_ID ='SS'
look_back=10
BOX=5
pitcher = prepro(TEAM_ID,pitcher16,hiter16)
x_train_single, y_train_single, x_val_single, y_val_single =split_data(pitcher.iloc[:,1:], look_back)
score, answer, pre = run(x_train_single, y_train_single, x_val_single, y_val_single, BOX)

Single window of past history : (10, 6)
Epoch 1/100
110/110 [==============================] - 62s 568ms/step - loss: 0.3578
Epoch 2/100
110/110 [==============================] - 1s 10ms/step - loss: 0.2643
Epoch 3/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2599
Epoch 4/100
110/110 [==============================] - 1s 10ms/step - loss: 0.2545
Epoch 5/100
110/110 [==============================] - 1s 10ms/step - loss: 0.2520
Epoch 6/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2503
Epoch 7/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2484
Epoch 8/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2474
Epoch 9/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2426
Epoch 10/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2381
Epoch 11/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2376
Epoch 12/100
110/110 [=====================

110/110 [==============================] - 1s 11ms/step - loss: 0.1609
Epoch 98/100
110/110 [==============================] - 1s 11ms/step - loss: 0.1597
Epoch 99/100
110/110 [==============================] - 1s 11ms/step - loss: 0.1582
Epoch 100/100
14/14 [==============================] - 22s 2s/step


In [16]:
score, answer, pre

(0.3339850604534149,
 array([0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0.]),
 array([[0.0923606 ],
        [0.79398793],
        [0.51625484],
        [1.0869662 ],
        [0.26938123],
        [0.59198076],
        [0.00178616],
        [0.2602263 ],
        [0.19490823],
        [0.6422416 ],
        [0.6925996 ],
        [0.5941944 ],
        [0.17130136],
        [0.64308774]], dtype=float32))

In [14]:
TEAM_ID ='LG'
look_back=10
BOX=5
pitcher = prepro(TEAM_ID,pitcher16,hiter16)
x_train_single, y_train_single, x_val_single, y_val_single =split_data(pitcher.iloc[:,1:], 10)
run(x_train_single, y_train_single, x_val_single, y_val_single,BOX)

Single window of past history : (10, 6)
Epoch 1/100
110/110 [==============================] - 63s 568ms/step - loss: 0.3280
Epoch 2/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2563
Epoch 3/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2552
Epoch 4/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2546
Epoch 5/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2529: 1
Epoch 6/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2512
Epoch 7/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2505
Epoch 8/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2498
Epoch 9/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2493
Epoch 10/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2499
Epoch 11/100
110/110 [==============================] - 1s 11ms/step - loss: 0.2495
Epoch 12/100
110/110 [==================

(0.24630995094776154,
 array([1. , 1. , 1. , 1. , 1. , 0.5, 0. , 1. , 0. , 0. , 1. , 0. , 1. ,
        0. ]),
 array([[0.46577033],
        [0.47173265],
        [0.4465012 ],
        [0.5577059 ],
        [0.5694852 ],
        [0.4372263 ],
        [0.4214755 ],
        [0.43892834],
        [0.42110103],
        [0.24577786],
        [0.08920822],
        [0.10243395],
        [0.31403732],
        [0.3896768 ]], dtype=float32))

In [9]:
loss = 0
team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']
loss_min=1000
for look_back in [5,10,20]:
    for box in [10, 20, 30,40]:
        loss = 0
        for team in team_list:
            pitcher = prepro(team ,pitcher16,hiter16)
            x_train_single, y_train_single, x_val_single, y_val_single =split_data(pitcher.iloc[:,1:], look_back)
            score, answer, pre = run(x_train_single, y_train_single, x_val_single, y_val_single, box)
            print(score, pre, answer)
            loss += score
        print(loss)
        if loss <loss_min:
            loss_min=loss
            box_min=box
            look_back_min=look_back
print(loss_min, box_min,look_back)
        


W0917 20:17:53.763159  8072 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 20:17:53.777178  8072 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0917 20:17:53.789171  8072 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 20:17:53.791175  8072 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Single window of past history : (5, 6)


W0917 20:17:54.386175  8072 deprecation.py:323] From F:\sda\envs\dlstudy\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0917 20:17:55.158174  8072 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0917 20:17:55.316174  8072 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/100
115/115 [==============================] - 1s 12ms/step - loss: 0.5087
Epoch 2/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2517
Epoch 3/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2506
Epoch 4/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2434
Epoch 5/100
115/115 [==============================] - 0s 2ms/step - loss: 0.2425
Epoch 6/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2446
Epoch 7/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2410
Epoch 8/100
115/115 [==============================] - 0s 2ms/step - loss: 0.2361
Epoch 9/100
115/115 [==============================] - 0s 2ms/step - loss: 0.2360
Epoch 10/100
115/115 [==============================] - 0s 2ms/step - loss: 0.2366
Epoch 11/100
115/115 [==============================] - 0s 2ms/step - loss: 0.2319
Epoch 12/100
115/115 [==============================] - 0s 2ms/step - loss: 0.2323
Epoch 13/100

115/115 [==============================] - 0s 2ms/step - loss: 0.1879
Epoch 100/100
19/19 [==============================] - 0s 7ms/step
0.2710973918437958 [[0.9229118 ]
 [0.9349582 ]
 [1.164846  ]
 [0.554958  ]
 [0.4990701 ]
 [0.72599524]
 [0.5723228 ]
 [0.4124465 ]
 [0.5420572 ]
 [0.5558126 ]
 [0.38563207]
 [0.54553187]
 [0.85850066]
 [0.91898614]
 [1.1175811 ]
 [0.6687883 ]
 [0.01333585]
 [0.14776802]
 [0.31128314]] [0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
Single window of past history : (5, 6)
Epoch 1/100
115/115 [==============================] - 1s 12ms/step - loss: 0.4551
Epoch 2/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2419
Epoch 3/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2250
Epoch 4/100
115/115 [==============================] - 0s 1ms/step - loss: 0.2265
Epoch 5/100
115/115 [==============================] - 0s 2ms/step - loss: 0.2266
Epoch 6/100
115/115 [==============================] - 0s 2ms/step -

...

100/100 [==============================] - 2s 16ms/step - loss: 0.2071
Epoch 33/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2075
Epoch 34/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2036: 0s - l
Epoch 35/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1997
Epoch 36/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2022
Epoch 37/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1930
Epoch 38/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1892
Epoch 39/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1945
Epoch 40/100
100/100 [==============================] - 2s 15ms/step - loss: 0.1852
Epoch 41/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1841
Epoch 42/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1852
Epoch 43/100
100/100 [==============================] - 2s 16ms/step - loss: 0.18

100/100 [==============================] - ETA: 0s - loss: 0.2194- ETA: 0s  - 2s 16ms/step - loss: 0.2208
Epoch 12/100
100/100 [==============================] - 2s 15ms/step - loss: 0.2131: 0
Epoch 13/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2189
Epoch 14/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2141
Epoch 15/100
100/100 [==============================] - 2s 17ms/step - loss: 0.2087
Epoch 16/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2084
Epoch 17/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2114
Epoch 18/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2075: 0s - l
Epoch 19/100
100/100 [==============================] - 2s 17ms/step - loss: 0.2020
Epoch 20/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2003
Epoch 21/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1997
Epoch 22/100
100/100 [=====================

100/100 [==============================] - 2s 16ms/step - loss: 0.1974
Epoch 30/100
100/100 [==============================] - 2s 15ms/step - loss: 0.1950
Epoch 31/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1933
Epoch 32/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1963
Epoch 33/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2036
Epoch 34/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1944
Epoch 35/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1896
Epoch 36/100
100/100 [==============================] - ETA: 0s - loss: 0.187 - 1s 15ms/step - loss: 0.1864
Epoch 37/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1842
Epoch 38/100
100/100 [==============================] - 2s 15ms/step - loss: 0.1824
Epoch 39/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1929
Epoch 40/100
100/100 [==============================] - 2s 16ms/s

100/100 [==============================] - 2s 17ms/step - loss: 0.1994
Epoch 24/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2072
Epoch 25/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1936
Epoch 26/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1923
Epoch 27/100
100/100 [==============================] - 2s 18ms/step - loss: 0.1878
Epoch 28/100
100/100 [==============================] - 2s 16ms/step - loss: 0.2078
Epoch 29/100
100/100 [==============================] - 2s 17ms/step - loss: 0.1840
Epoch 30/100
100/100 [==============================] - 2s 18ms/step - loss: 0.1837
Epoch 31/100
100/100 [==============================] - 2s 17ms/step - loss: 0.1861
Epoch 32/100
100/100 [==============================] - 2s 17ms/step - loss: 0.1778
Epoch 33/100
100/100 [==============================] - 2s 17ms/step - loss: 0.1839
Epoch 34/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1751
Epoch

100/100 [==============================] - 2s 20ms/step - loss: 0.2386 - ETA: 0s - loss: 0.2 - ETA: 0s - loss: 0.
Epoch 15/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2449
Epoch 00015: early stopping
4/4 [==============================] - 23s 6s/step
0.23435088992118835 [[0.6138087]
 [0.6345022]
 [0.5551755]
 [0.5212673]] [0. 1. 1. 1.]
3.865556165575981
2.6739945113658905 10 20


1.look_back 지정으로 데이터 손실문제
2.다른 변수들도 예측해야되는데 이게 되는건가
3. 각 팀별로 승 패 점수를 다르게 해야될듯 (labeling 할때 (승,무,패)(1,0.5,0)으로 지정)

In [11]:
print(loss_min, box_min,look_back_min)

2.6739945113658905 10 5


In [ ]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
def univariate(col, look_back):
    signal_data = pitcher16[[col]]
    # 데이터 전처리
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal_data = scaler.fit_transform(signal_data)

    # 데이터 분리
    train = signal_data[0:120]
    #val = signal_data[800:1200]
    test = signal_data[120:]

    # 데이터셋 생성
    x_train, y_train = create_dataset(train, look_back)
    #x_val, y_val = create_dataset(val, look_back)
    x_test, y_test = create_dataset(test, look_back)

    x_train = x_train.reshape(x_train.shape[0],look_back,1) #(size, timestamp,feature)
    x_test = x_test.reshape(x_test.shape[0],look_back,1)
    
    return x_train, x_test, y_train, y_test,scaler

In [ ]:
x_train, x_test, y_train, y_test,scaler = prepro("R",10)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping
look_back=10
model = Sequential()
model.add(Bidirectional(LSTM(20, input_shape=(look_back, 1)))) #(timestep,feature)
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

model.fit(x_train, y_train, epochs=100,
          batch_size=5, verbose=1, callbacks=[early_stop])

In [ ]:
model.evaluate(x_test,y_test,batch_size=5)

In [ ]:
pre =model.predict(x_test)
scaler.inverse_transform(pre)


In [ ]:
scaler.inverse_transform(pre).mean()

In [ ]:
scaler.inverse_transform(y_test.reshape(14,1))

In [ ]:
scaler.inverse_transform(y_test.reshape(14,1)).mean()

In [ ]:
target_minmax.shape

In [ ]:
signal_data

In [ ]:
signal_data =np.concatenate((signal_data,target_minmax), axis=1)

In [ ]:
signal_data[:,-1]